This is an IPython notebook to keep track of various results. Contained in this notebook are various graphs and data points, as well as the hyperparameters and implementations that generated these results.

<h1>MNIST Classification</h1>
The first experiment we have worked with is incrementally trying to replicate the results from <a href=http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf> this paper.</a> The paper focuses on semi-supervised learning for a small neural network to classify MNIST digits based on only a small amount of labeled data.

We started with implementing a very simple network that had the inputs mapped directly to the output, with a softmax activation at the end, and the loss function is the cross entropy. The input size was a 1x784 vector of image data. The results were the following:

| Batch Size | Iterations | Accuracy |
|------------|------------|----------|
| 60000      | 300        | 91.34%   |
| 100        | 10000      | 92.12%   |
| 1          | 100000     | 89.57%   |

Next, we moved onto a more complex model.  This one had a single hidden layer, which had dropout on it, and was trained on the same 600 examples in batch sizes of 32. This was so as to begin replication of the results in the paper.  This architecture matches the dropNN as specified in the paper (with the exception of the optimizer - we used Adam, which is a similar momentum based SGD algorithm)

The paper suggests using a hidden layer of size 5000. Instead, we opted to try various layer sizes to see how that affected both accuracy and conditional entropy (which is another measure of certainty of the classifier). AS for iterations, I used 10000 for both 600 examples, and all examples with batch size of 32.  For batch size of 1, I used 60000 iterations.  The results are as follows, starting with conditional entropy:

In [1]:
import plotly.plotly as pl
import plotly.graph_objs as go

# Read in data
lsize = []
ce = []
with open('code/xe_dropNN.txt') as f:
    val = f.readline()
    while val:
        ce.append(float(val))
        val = f.readline()
        
with open('code/lsize.txt') as f:
    val = f.readline()
    while val:
        lsize.append(int(val))
        val = f.readline()

ce_nb = []
with open('code/xe_dropNN_nb.txt') as f:
    val = f.readline()
    while val:
        ce_nb.append(float(val))
        val = f.readline()
        
ce_1 = []
with open('code/xe_dropNN_1.txt') as f:
    val = f.readline()
    while val:
        ce_1.append(float(val))
        val = f.readline()

trace0 = go.Scatter(x = lsize, y = ce, mode = 'lines', name='600 examples')
trace1 = go.Scatter(x = lsize, y = ce_nb, mode='lines', name='Every example, 32')
trace2 = go.Scatter(x = lsize, y = ce_1, mode='lines', name='Every example, 1')
layout = go.Layout(title='Conditional Entropy vs Layer Size')
fig = go.Figure(data=[trace0,trace1,trace2], layout=layout)
pl.iplot(fig, filename='basic-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~liamcwalsh/0 or inside your plot.ly account where it is named 'basic-scatter'


In [3]:
import plotly.plotly as pl
import plotly.graph_objs as go

# Read in data
lsize = []
acc_600 = []
with open('code/acc_dropNN.txt') as f:
    val = f.readline()
    while val:
        acc_600.append(float(val))
        val = f.readline()
        
with open('code/lsize.txt') as f:
    val = f.readline()
    while val:
        lsize.append(int(val))
        val = f.readline()

acc_32 = []
with open('code/acc_dropNN_nb.txt') as f:
    val = f.readline()
    while val:
        acc_32.append(float(val))
        val = f.readline()

acc_1 = []
with open('code/acc_dropNN_1.txt') as f:
    val = f.readline()
    while val:
        acc_1.append(float(val))
        val = f.readline()

trace0 = go.Scatter(x = lsize, y = acc_600, mode = 'lines', name='600 examples')
trace1 = go.Scatter(x = lsize, y = acc_32, mode='lines', name='Every ex, 32')
trace2 = go.Scatter(x = lsize, y = acc_1, mode='lines', name='Every ex, 1')
layout = go.Layout(title='Accuracy vs Layer Size')
fig2 = go.Figure(data=[trace0, trace1, trace2], layout=layout)
pl.iplot(fig2, filename='basic-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~liamcwalsh/0 or inside your plot.ly account where it is named 'basic-scatter'


<h1>Pseudolabels</h1>
<h2>Hyperparameters</h2>
Drop prob: .5

Hidden layer: 300 units

Batch size: 32 labeled, 256 unlabeled

Weighting scheme - Same as paper, but T1=200
<h2>Results</h2>
First, we looked at using 300 hidden units to classify.  At each iteration, new data was provided, some of which was marked as unlabeled. Below are the results and hyperparams used:

| Iterations | Accuracy - PL | Accuracy - No PL |
|------------|---------------|------------------|
| 1000       | 41.12%        | 45%              |
| 5000       | 82.57%        | 81.35%           |
| 10000      | 87.75%        | 87.26%           |  


The purpose of pseudolabels is to help with classification when there are far fewer data points, regardless of number of iterations.  This is what we examined next.  Using <strong>only 600 labeled examples, and 6000 pseudolabels</strong>, the results are as follows:

| Iterations | Accuracy - PL | Accuracy - No PL |
|------------|---------------|------------------|
| 1000       |     43.93%    |    47.91%        |
| 5000       |     78.93%    |    79.12%        |
| 10000      |     84.78%    |    84.62%        |


<h2>Hyperparameters</h2>
Drop prob: .5

Hidden layers: 5000

Batch size: 32 labeled, 256 unlabeled

<h2>Results</h2>
Next, we can look at a larger number of hidden units.  The paper uses 5000. As a baseline, we first train with all the data, using batch sizes of 32 labeled, 256 unlabeled, and drop prob of .5

| Iterations | Accuracy - PL | Accuracy - No PL |
|------------|---------------|------------------|
| 1000       | 79.66%        | 80.76%           |
| 5000       | 90.08%        | 89.78%           |

Next, again using only 600 labeled, 6000 unlabeled:

| Iterations | Accuracy - PL | Accuracy - No PL |
|------------|---------------|------------------|
| 1000       | 79.89%        | 80.76%           |
| 5000       | 86.02%        | 89.78%           |


<h1>Varying learning rate</h1>
In the pseudolabel loss, there is a learning rate that modifies how much the total loss should take into account the error on the pesudolabels.  This starts at 0 for the first few iterations, then increases with time, until it hits some max value alpha_f.  Here, we again use the 600-6000 split for the training, which is about a 9-91 split of the data. Using 600 hidden units, and the same batch sizes (32, 256, for a 1:8 ratio), and 5000 iterations, we vary the learning rate by .1 from .1 up to 10

<h1>Minimizing params</h1>
To get a better understanding, we want to try to minimize the params of the model as much as we can.  Batch size of 1.

To start as a benchmark, the single layer was able to get 96% acc (peaked about 97%, but this was closer to 5000 units, and only marginal improvement upon 96%, so close that it could be chance) with 1000 hidden units. We want to see the smallest 2 layer that can hit that benchmark

200 units total (100 and 100) yielded 96% accuracy. To get a better sense of how layer sizes interact, I tried adjusting the relative sizes to see effects on accuracy:


| Layer 1    | Layer 2       | Accuracy         |
|------------|---------------|------------------|
| 175        | 25            | 96.12%           |
| 150        | 50            | 96.1%            |
| 100        | 100           | 96.13%           |
| 50         | 150           | 95.44%           |
| 25         | 175           | 93.95%           |

85, 80 seems to be the hard lower bound - all else can't crack 96%

300, 100. batch = 60000, iter = 100: 83.51%

300, 100. batch = 60000, iter = 500: 94.45%

On the LeCun page, for a small 2 layer NN, they found an accuracy of 96.95% for 300,100 hidden units. 

| Layer 1    | Layer 2       | Accuracy         |
|------------|---------------|------------------|
| 300        | 100           | 97.13%           |
| 290        | 100           | 97%              |
| 275        | 100           | 96.61%           |
| 250        | 125           | 96.97%           |
| 250        | 100           | 96.34%           |

With convolution, the simplest architecture (LeNet-1) has an accuracy of 98.3%. Modifying any aspect of this (which is just a series of filters) results in a drop in accuracy - turns out the godfather of NN knows what he's doing.

Here, I have 1 conv layer (5x5 filter), a max pool layer, and then 2 hidden layers. For 10 filters:


| Layer 1    | Layer 2       | Accuracy         |
|------------|---------------|------------------|
| 30         | 20            | 95.21%           |
| 40         | 30            | 96.78%           |
| 50         | 30            | 97.23%           |
| 50         | 50            | 97.54%           |
| 90         | 90            | 97.67%           |
| 100        | 100           | 97.70%           |
| 200        | 200           | 98.2%            |
| 500        | 200           | 98.61%           |

We want to see how we can shrink the parameters and still hit the 96.95% accuracy mark. This is with 5 filters:

| Layer 1    | Layer 2       | Accuracy         |
|------------|---------------|------------------|
| 100        | 50            | 97.38%           |
| 200        | 100           | 97.64%           |
| 90         | 50            | 96.55%           |
| 30         | 30            | 95.85%           |
